# Tutorial: Analysis (work in progress)

## Initial imports

In [1]:
from datetime import datetime
from straklip import config, input_tables
from straklip.steps import analysis
from straklip.stralog import getLogger
import os
import pkg_resources as pkg

First, we need to initialize the logger here.

In [2]:
if 'SHARED_LOG_FILE' not in os.environ:
    os.environ['SHARED_LOG_FILE'] = f'straklip_{datetime.now().strftime("%Y-%m-%d_%H%M")}.log'

getLogger('straklip', setup=True, logfile=os.environ['SHARED_LOG_FILE'],debu=False,
          configfile=pkg.resource_filename('straklip', './config/logging.yaml'))

<Logger straklip (DEBUG)>

Two pipeline configuration files are stored in the `tutorials/pipeline_logs`, directory, namely `pipe.yaml` and `data.yaml`. A more in-deep explanation of these file, is presented here: https://straklip.readthedocs.io/latest/quick_start.html. We start by loading into the pipeline as follow. Remember, to manipulate the options for each specific step, we can change the entries in the 'pipe.yaml' accordingly, or change them from a line of code like `pipe_cfg.analysis['steps']['extract_candidate']=True`,  before running the step.
For this tutorial we will start form the previously generated dataframes from the pipeline.

In [3]:
pipe_cfg='/Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/pipeline_logs/pipe.yaml' #or where these files are
data_cfg='/Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/pipeline_logs/data.yaml'
pipe_cfg = config.configure_pipeline(pipe_cfg,pipe_cfg=pipe_cfg,data_cfg=data_cfg,dt_string=datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
data_cfg = config.configure_data(data_cfg,pipe_cfg)

2025-06-19 09:29:28 straklip.config             :INFO     (configure_pipeline:70[pid=13110]) 
StraKLIP pipeline started at date and time: 19/06/2025 09:29:28
Pipe_cfg: /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/pipeline_logs/pipe.yaml
Data_cfg: /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/pipeline_logs/data.yaml

2025-06-19 09:29:28 straklip.config             :INFO     (configure_data:151[pid=13110]) Validation of default labels and data successful!


## Loading the dataframe

Once the "pipe_cfg" and the "data_cfg" are configuration, we can load pre-existing dataframes.

In this case we use "skip_originals" True and "load" True to tell the pipeline tom not look for inpout catalogs, but for the dataframe already generated by the pipeline.

NOTE: for this tutorial, we are forcing the pipeline to work on only the "unq_id" 52 to speed up the process, as follow (you can achieve the same result by changing the "unq_ids_list" option in the "pipe.yaml"):

In [4]:
dataset = input_tables.Tables(data_cfg, pipe_cfg, skip_originals=True)
DF = config.configure_dataframe(dataset,load=True)

2025-06-19 09:29:29 straklip.config             :WARNING  (configure_dataframe:224[pid=13110]) get_Av_dict currently only supports VEGAmag system. Please provide your own set of AVs if in a differest system as AVs : {ext: {mag_filter : value}} in the data.yaml under target
2025-06-19 09:29:29 straklip.utils.ancillary    :INFO     (get_Av_dict:424[pid=13110]) before dust, V =  0.0 mag(VEGA)
2025-06-19 09:29:29 straklip.utils.ancillary    :INFO     (get_Av_dict:425[pid=13110]) after dust, V = 1.0146 mag(VEGA)
2025-06-19 09:29:29 straklip.utils.ancillary    :INFO     (get_Av_dict:444[pid=13110]) Av = 1.0146 mag


2025-06-19 09:29:30 straklip.utils.ancillary    :INFO     (get_Av_dict:488[pid=13110]) AV=0 wfc3,uvis2,f814w 0.0 mag(VEGA)
2025-06-19 09:29:30 straklip.utils.ancillary    :INFO     (get_Av_dict:489[pid=13110]) AV=1 wfc3,uvis2,f814w 0.6094 mag
2025-06-19 09:29:30 straklip.utils.ancillary    :INFO     (get_Av_dict:488[pid=13110]) AV=0 wfc3,uvis2,f850lp 0.0 mag(VEGA)
2025-06-19 09:29:30 straklip.utils.ancillary    :INFO     (get_Av_dict:489[pid=13110]) AV=1 wfc3,uvis2,f850lp 0.4694 mag
2025-06-19 09:29:30 straklip.config             :INFO     (configure_dataframe:270[pid=13110]) Fetching dataframes from /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out


In [5]:
DF.keys

['unq_targets',
 'crossmatch_ids',
 'mvs_targets',
 'mvs_candidates',
 'unq_candidates']

In [6]:
DF.mvs_targets_df

,mvs_ids,x_f814w,x_f850lp,y_f814w,y_f850lp,vis,ext,counts_f814w,counts_f850lp,ecounts_f814w,...,e_f814w_ap,counts_f850lp_ap,ecounts_f850lp_ap,nsky_f850lp_ap,m_f850lp_ap,e_f850lp_ap,delta_f814w,edelta_f814w,delta_f850lp,edelta_f850lp
0,0,766.682062,766.297865,869.519962,870.863230,13,1,10050.176567,8147.884608,103.716767,...,0.015694,5112.833,75.324845,25.0,21.159546,0.017460,24.618,0.007,23.3,0.007
1,1,769.100793,768.870891,866.146099,866.488792,1,1,37148.970247,29523.416694,200.603377,...,0.010132,18404.541,140.319831,25.0,19.768888,0.010841,24.618,0.007,23.3,0.007
2,2,762.415167,762.213142,869.798303,870.180384,2,1,20888.773322,16756.185268,150.306225,...,0.012022,10545.567,107.937060,25.0,20.379608,0.013133,24.618,0.007,23.3,0.007
3,3,765.630228,765.408206,867.979551,868.467341,3,1,19851.347253,17782.298850,146.991839,...,0.012219,11221.743,109.214962,25.0,20.306049,0.012675,24.618,0.007,23.3,0.007
4,4,767.251059,767.009600,867.473929,867.939866,4,1,18535.997643,17143.439224,141.164681,...,0.012490,10739.130,107.621370,25.0,20.353777,0.012938,24.618,0.007,23.3,0.007
5,5,766.601507,766.336710,868.267308,868.601284,5,1,21899.906087,19840.838076,152.199777,...,0.011737,12351.680,115.865474,25.0,20.207967,0.012358,24.618,0.007,23.3,0.007
6,6,767.081804,766.867054,866.901849,867.166727,6,1,10821.808262,9736.723197,111.510075,...,0.015567,6046.163,81.943106,25.0,20.977500,0.016295,24.618,0.007,23.3,0.007
7,7,765.825833,765.608929,868.625604,869.098238,7,1,14345.541776,12125.508627,123.726601,...,0.013657,7633.587,90.881860,25.0,20.724378,0.014700,24.618,0.007,23.3,0.007
8,8,765.740929,765.619928,868.295724,868.761725,8,1,12134.206337,9881.013676,116.470224,...,0.014754,6125.534,82.390402,25.0,20.963340,0.016194,24.618,0.007,23.3,0.007
9,9,762.743761,762.620949,866.727766,866.796443,9,1,12163.774100,9674.484288,118.946485,...,0.014918,6044.626,82.730407,25.0,20.977776,0.016426,24.618,0.007,23.3,0.007


## Running the analysis

The next step wil perform a more in depth analysis of each candidate. Manipulating the options in the "pipe.yaml" under the "analysis" block we can perform raw contrast curves, calibrated contrast curves, the extraction of a candidate though forward modeling and MCMC fitting.

### Candidate Extraction

Note in this example we limit the analysis to just one target, setting the use unq_ids_list: [52].

In [7]:
pipe_cfg.unq_ids_list = [52]

 In the following we force the pipeline to just run  the extract_candidate step, and we ensure that all the other step of the analysis are turned off.

In [12]:
pipe_cfg.analysis['steps']['extract_candidate']=True
pipe_cfg.analysis['steps']['contrast_curves']=False
pipe_cfg.analysis['steps']['cal_contrast_curves']=False
pipe_cfg.analysis['steps']['mass_sensitivity_curves']=False
analysis.run({'DF': DF, 'dataset': dataset})

2025-06-19 09:10:03 straklip.steps.analysis     :INFO     (run:1450[pid=89762]) Running Analysis step
2025-06-19 09:10:03 straklip.config             :INFO     (make_paths:115[pid=89762]) "/Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/analysis/" exists, and will not be created.
2025-06-19 09:10:03 straklip.steps.analysis     :INFO     (setup_DATASET:87[pid=89762]) Setting up Dataset for observation
2025-06-19 09:10:03 straklip.steps.analysis     :INFO     (generate_psflib:108[pid=89762]) Generating PSF library
2025-06-19 09:10:04 straklip.steps.analysis     :INFO     (candidate_extraction:914[pid=89762]) Extracting candidate from: ['/Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/mvs_tiles/f814w/tile_ID52.fits']
2025-06-19 09:10:04 straklip.steps.analysis     :WARNING  (run_analysis:1444[pid=89762]) Cannot convert contrast curves in mass sensistivity curves without an interpolator!
2025-06-19 09:10:04 straklip.steps.analysis     :INFO     (setup_DATASET:87

### Raw Contrast Curves

In [13]:
pipe_cfg.analysis['steps']['extract_candidate']=False
pipe_cfg.analysis['steps']['contrast_curves']=True
pipe_cfg.analysis['steps']['cal_contrast_curves']=False
pipe_cfg.analysis['steps']['mass_sensitivity_curves']=False
analysis.run({'DF': DF, 'dataset': dataset})

2025-06-19 09:10:14 straklip.steps.analysis     :INFO     (run:1450[pid=89762]) Running Analysis step
2025-06-19 09:10:14 straklip.config             :INFO     (make_paths:115[pid=89762]) "/Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/analysis/" exists, and will not be created.
2025-06-19 09:10:14 straklip.steps.analysis     :INFO     (setup_DATASET:87[pid=89762]) Setting up Dataset for observation
2025-06-19 09:10:14 straklip.steps.analysis     :INFO     (generate_psflib:108[pid=89762]) Generating PSF library
2025-06-19 09:10:15 straklip.steps.analysis     :INFO     (mk_contrast_curves:1058[pid=89762]) Making contrast curves.
2025-06-19 09:10:15 straklip.steps.analysis     :INFO     (mk_contrast_curves:1062[pid=89762]) Loading candidate dictionary from file: /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/analysis/ID52/extracted_candidate/f814w_extracted.yaml
2025-06-19 09:10:15 straklip.steps.analysis     :INFO     (mk_contrast_curves:1065[pid=89762]) Lo

  0%|          | 0/1 [00:00<?, ?it/s]

2025-06-19 09:10:20 straklip.steps.analysis     :INFO     (mk_contrast_curves:1071[pid=89762]) Loading residuas from file: /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/analysis/ID52/masked_candidates/f814w-res_masked-KLmodes-all.fits
2025-06-19 09:10:20 straklip.steps.analysis     :INFO     (mk_contrast_curves:1112[pid=89762]) Saving contrast curves plot in: /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/analysis/ID52
2025-06-19 09:10:20 straklip.steps.analysis     :INFO     (mk_contrast_curves:1136[pid=89762]) residual masked plot in: /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/analysis/ID52
2025-06-19 09:10:20 straklip.steps.analysis     :INFO     (mk_contrast_curves:1146[pid=89762]) Saving contrast curves dictionary to file: /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/analysis/ID52/f814w_contrast_curves.pkl
2025-06-19 09:10:20 straklip.steps.analysis     :WARNING  (run_analysis:1444[pid=89762]) Cannot convert contra

  0%|          | 0/1 [00:00<?, ?it/s]

2025-06-19 09:10:25 straklip.steps.analysis     :INFO     (mk_contrast_curves:1071[pid=89762]) Loading residuas from file: /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/analysis/ID52/masked_candidates/f850lp-res_masked-KLmodes-all.fits
2025-06-19 09:10:25 straklip.steps.analysis     :INFO     (mk_contrast_curves:1112[pid=89762]) Saving contrast curves plot in: /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/analysis/ID52
2025-06-19 09:10:26 straklip.steps.analysis     :INFO     (mk_contrast_curves:1136[pid=89762]) residual masked plot in: /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/analysis/ID52
2025-06-19 09:10:26 straklip.steps.analysis     :INFO     (mk_contrast_curves:1146[pid=89762]) Saving contrast curves dictionary to file: /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/analysis/ID52/f850lp_contrast_curves.pkl
2025-06-19 09:10:26 straklip.steps.analysis     :WARNING  (run_analysis:1444[pid=89762]) Cannot convert cont

### Calibrated Contrast Curves

In [9]:
pipe_cfg.analysis['steps']['extract_candidate']=False
pipe_cfg.analysis['steps']['contrast_curves']=False
pipe_cfg.analysis['steps']['cal_contrast_curves']=True
pipe_cfg.analysis['steps']['mass_sensitivity_curves']=False
analysis.run({'DF': DF, 'dataset': dataset})

2025-06-19 09:36:18 straklip.steps.analysis     :INFO     (run:1453[pid=13110]) Running Analysis step
2025-06-19 09:36:18 straklip.config             :INFO     (make_paths:115[pid=13110]) "/Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/out/analysis/" exists, and will not be created.
2025-06-19 09:36:18 straklip.steps.analysis     :INFO     (setup_DATASET:87[pid=13110]) Setting up Dataset for observation
2025-06-19 09:36:18 straklip.steps.analysis     :INFO     (generate_psflib:108[pid=13110]) Generating PSF library
2025-06-19 09:36:19 straklip.steps.analysis     :INFO     (mk_cal_contrast_curves:1151[pid=13110]) Making corrected contrast curves.
2025-06-19 09:36:19 straklip.steps.analysis     :INFO     (mk_cal_contrast_curves:1154[pid=13110]) Loading masked candidate data.
2025-06-19 09:36:19 straklip.steps.analysis     :INFO     (mk_cal_contrast_curves:1163[pid=13110]) Loading contrast curves dictionary from file: /Users/gstrampelli/PycharmProjects/StraKLIP_tutorial_test/ou

  0%|          | 0/1 [00:00<?, ?it/s]

2025-06-19 09:36:23 straklip.steps.analysis     :INFO     (mk_cal_contrast_curves:1188[pid=13110]) Injecting fake 2 at pa: 45, sep: 0.922, flux: 72154844.11843841


  0%|          | 0/1 [00:00<?, ?it/s]

2025-06-19 09:36:27 straklip.steps.analysis     :INFO     (mk_cal_contrast_curves:1188[pid=13110]) Injecting fake 3 at pa: 90, sep: 0.922, flux: 72154844.11843841


...

  0%|          | 0/1 [00:00<?, ?it/s]

2025-06-19 09:39:05 straklip.steps.analysis     :INFO     (mk_cal_contrast_curves:1188[pid=13110]) Injecting fake 41 at pa: 0, sep: 5.532, flux: 165.59019252274615


  0%|          | 0/1 [00:00<?, ?it/s]

Process SpawnPoolWorker-1356:
Process SpawnPoolWorker-1353:
Process SpawnPoolWorker-1355:
Process SpawnPoolWorker-1354:
Process SpawnPoolWorker-1352:
Process SpawnPoolWorker-1349:
Process SpawnPoolWorker-1350:
Process SpawnPoolWorker-1348:
Process SpawnPoolWorker-1351:
Process SpawnPoolWorker-1347:
Process SpawnPoolWorker-1346:
Process SpawnPoolWorker-1344:
Process SpawnPoolWorker-1345:
Process SpawnPoolWorker-1343:
Process SpawnPoolWorker-1342:
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/miniconda3/envs/straKLIP/lib/python3.11/multiprocessing/spawn.py", line 120, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/straKLIP/lib/python3.11/multiprocessing/spawn.py", line 128, in _main
Process SpawnPoolWorker-1357:
    preparation_data = reduction.pickle.load(from_parent)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
Traceback (most recent ca

KeyboardInterrupt: 

### Contrast to mass conversion (work in progress)

To convert contrast to masses, we use user provided interpolator that will transform (mag,age) -> (mass), (mag,age) -> (teff), (mag,age) -> (logR). This interpolator should be assemble as a series of dictionaries such as interp = {filter: {'logmass': (mag,age) -> (mass), 'teff': (mag,age) -> (teff), 'logR': (mag,age) -> (logR)}}.
First we convert the contrast for detect companion

In [ ]:
pipe_cfg.analysis['path2iso_interp']='path/to/user/interpolator.pkl'
pipe_cfg.analysis['steps']['extract_candidate']=True
pipe_cfg.analysis['steps']['contrast_curves']=False
pipe_cfg.analysis['steps']['cal_contrast_curves']=False
pipe_cfg.analysis['steps']['mass_sensitivity_curves']=False
analysis.run({'DF': DF, 'dataset': dataset})

Then we convert the calibrate contrast curves

In [ ]:
pipe_cfg.analysis['path2iso_interp']='path/to/user/interpolator.pkl'
pipe_cfg.analysis['steps']['extract_candidate']=False
pipe_cfg.analysis['steps']['contrast_curves']=False
pipe_cfg.analysis['steps']['cal_contrast_curves']=False
pipe_cfg.analysis['steps']['mass_sensitivity_curves']=True
analysis.run({'DF': DF, 'dataset': dataset})